In [ ]:
using Pkg, CairoMakie, LaTeXStrings, LinearAlgebra, FFTW
Pkg.activate("/Users/jiyong/.juliaenv/image")
using OpenCV, TestImages
const cv = OpenCV

In [ ]:
include("cvext.jl")

In [ ]:
fig = Figure()
ax1 = Axis(fig[1,1], title=L"f(t)", limits = (-5, 5, 0, 7), xticks=([-1, 0, 1], [L"-T", L"0",  L"T"]), yticks=([], []), xgridvisible = false, ygridvisible = false, xlabel = L"t")
f = x->0.5*exp(-(x+4.5)^2/0.3) + 5.0*exp(-(x+1)^2/0.7) + 4.0*exp(-(x-1)^2/2) + 2.0*exp(-(x-4)^2/2) +0.4*exp(-(x-7.5)^2/2)+0.1
t=-5:0.1:5
lines!(ax1, t, f.(t))
t2 = -5:1:5
for i in t2
    lines!(ax1, [i, i], [0, 1], color=:red)
end

ax2 = Axis(fig[2, 1], title = L"f_I(t)", limits = (-5, 5, 0, 7), xticks=([-1, 0, 1], [L"-T", L"0",  L"T"]), yticks=([], []), xgridvisible = false, ygridvisible = false, xlabel = L"t")

scatter!(ax2, t2, f.(t2))
#save("sampling.png", fig)
fig

In [ ]:
struct skew
    x0::Float64
    w::Float64
end

function (p::skew)(x)
    x0, w= p.x0, p.w
    
    if abs(x-x0) > w
        return 0.0
    
    elseif x>x0
        return -1.0/w*(x-x0)+1
    else
        return 1.0/w*(x-x0)+1
    return y
    end
end

p = skew(0, 1)
fig = Figure(size=(400, 600))
ax1 = Axis(fig[1,1], title=L"\hat{f}(\mu)", limits=(-5, 5, -0.1, 1.1), xticks = ([-1, 0,  1], [L"-\mu_M", L"0", L"\mu_M"]), yticks=([0, 1]), xgridvisible = false, ygridvisible = false)
u0 = -5:0.01:5
lines!(ax1, u0, p.(u0))
text!(ax1, L"(a)", position=(-4.8,0.8))

ax2 = Axis(fig[2,1], title=L"\hat{f}_s(\mu)", limits=(-5, 5, -0.1, 1.1),xticks = ([-2.5,0,  2.5], [L"-1/T",L"0", L"1/T"]), yticks=([0, 1]), xgridvisible = false, ygridvisible = false)
q(x) = skew(0, 1)(x) + skew(2.5, 1)(x) + skew(-2.5, 1)(x) + skew(5, 1)(x) + skew(-5, 1)(x)
t1=-5:0.01:5
lines!(ax2, t1, q.(t1))
text!(ax2, L"(b)", position=(-4.8,0.8))

ax3 = Axis(fig[3,1], title=L"\hat{f}_s(\mu)", limits=(-5, 5, -0.1, 1.1), xticks = ([-1,0,  1], [L"-1/T", L"0", L"1/T"]),  yticks=([0, 1]), xgridvisible = false, ygridvisible = false)
q(x) = skew(0, 1)(x) + skew(2, 1)(x) + skew(-2, 1)(x) + skew(4, 1)(x) + skew(-4, 1)(x)
t2=-5:0.01:5
lines!(ax3, t2, q.(t2))
text!(ax3, L"(c)", position=(-4.8,0.8))

ax4 = Axis(fig[4,1], title=L"\hat{f}_s(\mu)",limits=(-5, 5, -0.1, 1.1),  xticks = ([-1.7,0,  1.7], [L"-1/T",L"0", L"1/T"]),  yticks=([0, 1]), xgridvisible = false, ygridvisible = false)
q(x) = skew(0, 1)(x) + skew(1.7, 1)(x) + skew(-1.7, 1)(x) + skew(3.4, 1)(x) + skew(-3.4, 1)(x) +skew(5.1, 1)(x) + skew(-5.1, 1)(x)
t3 = -5:0.01:5
lines!(ax4, t3, q.(t3))
text!(ax4, L"(d)", position=(-4.8,0.8))

fig
#save("sample_freqdomain.png", fig)

In [ ]:
t=-1:0.1:1
lines(imag.(fftshift(fft(p.(t)))))

In [ ]:
cv.Mat(ft1.data[2:2,:,:]);

In [ ]:
cc = fft(img1a)[:,:,1]
cvx.arr2mat(real(cc));

In [ ]:
function a2m(arr::Array{T, 3}) where T<:Union{Float32, Float64, Int16, Int32, Int8, UInt16, UInt8}
    OpenCV.Mat(permutedims(arr, [3,2,1]))
end

X = zeros(UInt8, (256, 512))
X[(128-5):(128+5), (256-15):(256+15)].= 255
img1 = cvx.arr2mat(X)
img1f = cvx.ConvertTo(img1, Float32)
img2 = cv.dft(img1f, flags=cv.DFT_COMPLEX_OUTPUT);
img22 = cvx.fftshift(img2)
arr2 = cvx.mat2arr(img2)
img1


In [ ]:
cv.split(img2)[1]

In [ ]:
heatmap(log.(1.0 .+ sqrt.(arr2[:,:,1].^2 + arr2[:,:,2].^2)))

In [ ]:
Xf0 = fft(X')
Xf = fftshift(Xf0)
f = Figure(size=(800, 400))
ax1 = Axis(f[1, 1], aspect=DataAspect(), yreversed = true, title = L"X")
hm1 = heatmap!(ax1, X')
ax2 = Axis(f[1, 2], aspect=DataAspect(), yreversed = true, title=L"\mathfrak{F}[X]")
hm2 = heatmap!(ax2, log.(1.0 .+ abs.(Xf)))
ax3 = Axis(f[2, 1], aspect=DataAspect(), yreversed = true, title = L"\text{Re}(\mathfrak{F}[X])")
hm3 = heatmap!(ax3, log.(1.0 .+ abs.(real(Xf))))
ax4 = Axis(f[2, 2], aspect=DataAspect(), yreversed = true, title = L"\text{Im}(\mathfrak{F}[X])")
hm4 = heatmap!(ax4, log.(1.0 .+ abs.(imag(Xf))))
f


In [ ]:
heatmap(log.(1.0 .+ abs.(Xf)))


In [ ]:

fig, ax, hm = heatmap(log.(1.0 .+ abs.(fftshift(real(fft(X))))))
Colorbar(fig[:, end+1], hm)
fig

In [ ]:

fig, ax, hm = heatmap(log.(1.0 .+ abs.(real(fft(X)))))
Colorbar(fig[:, end+1], hm)
fig

In [ ]:

function tifftshift(v::Vector)
    m = length(v)
    n = iseven(m) ? div(m, 2) : div(m, 2)
    return [v[n+1:end]; v[1:n] ]
end 


function tifftshift(A::Matrix)
    h, w = size(A)
    m = iseven(h) ? div(h, 2) : div(h, 2)
    n = iseven(w) ? div(w, 2) : div(w, 2)
    return [A[m+1:end, n+1:end] A[m+1:end, 1:n] ; A[1:m, n+1:end] A[1:m, 1:n] ]
end



In [ ]:
ps = reshape(collect(Int32, 1:64), (8, 8))
ifftshift(ps)

In [ ]:
tifftshift(ps)

In [ ]:
tfftshift(p)

In [ ]:
tfftshift(p)[1,:,:]

In [ ]:
cvx.fftshift(ft1)

In [ ]:
function tfftshift(A::OpenCV.Mat)
    
    # to be imporved by reducing array allocation
    return arr2mat(fftshift(mat2arr(A)[:,:,1]))
end

In [ ]:
size(ft1)

In [ ]:
#permutedims(stack([arr, ]), [3,2,1]))
xx = OpenCV.Mat(permutedims(stack([cvx.fftshift(cvx.mat2arr(ft1)[:, :, i]) for i in 1:size(img2)[1]], dims=3), [3,2,1]))
yy = sqrt.(xx[1,:,:].^2 .+ xx[2,:,:,].^2)
log.(1.0 .+ yy)
fig, ax, hm = heatmap(log.(1.0 .+ yy))
Colorbar(fig[:, end+1], hm)
fig

In [ ]:
xx

In [ ]:
cvx.mat2arr(ft1[1, :, :])

In [ ]:
size(img0)

In [ ]:
size(img22)

In [ ]:
ps=cv.imread("../imgs/ero-adbox-gallery-2020-033a-2400x1200.jpg");


In [ ]:
size(ps)

In [ ]:
xq=cv.split(ps)

In [ ]:
cv.merge(OpenCV.InputArray[xq[1],xq[2], xq[3]]);

In [ ]:
cv.dft(cvx.ConvertTo(ps, Float32), flags=cv.DFT_COMPLEX_OUTPUT);

In [ ]:
cvx.ConvertTo(ps, Float32);

In [ ]:
ps.data

In [ ]:
OpenCV.InputArray[xq[1],xq[2], xq[3]]

In [ ]:
mod(3.0, 0)